## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-30-02-43-36 +0000,bbc,Trump says 'very dangerous' for UK to deal wit...,https://www.bbc.com/news/articles/c0keyyeyr41o...
1,2026-01-30-02-37-40 +0000,bbc,Trump and his sons sue IRS and US Treasury ove...,https://www.bbc.com/news/articles/c23ryyrx40yo...
2,2026-01-30-02-37-00 +0000,wsj,Some Minnesotans Celebrate Victory as Border C...,https://www.wsj.com/us-news/some-minnesotans-c...
3,2026-01-30-02-36-16 +0000,bbc,Suspected jihadists attack airport near Niger'...,https://www.bbc.com/news/articles/cvgp47neze4o...
4,2026-01-30-02-35-34 +0000,nyt,Video Shows Bovino’s Orders to Immigration Age...,https://www.nytimes.com/2026/01/29/us/bovino-o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2444/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,54
96,ice,23
310,new,19
87,he,17
58,iran,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
26,2026-01-30-01-53-00 +0000,wsj,President Trump and Senate Democrats said ther...,https://www.wsj.com/politics/policy/white-hous...,153
372,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,126
159,2026-01-29-20-34-23 +0000,nypost,Trump prevents ‘boring’ cabinet meeting so he ...,https://nypost.com/2026/01/29/us-news/trump-sp...,117
86,2026-01-29-23-30-25 +0000,nyt,"Democrats Reach Spending Deal With Trump, Seek...",https://www.nytimes.com/2026/01/29/us/politics...,110
47,2026-01-30-00-48-14 +0000,nyt,Trump Weighs New Military Options Against Iran,https://www.nytimes.com/2026/01/29/us/trump-mi...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
26,153,2026-01-30-01-53-00 +0000,wsj,President Trump and Senate Democrats said ther...,https://www.wsj.com/politics/policy/white-hous...
290,85,2026-01-29-13-31-00 +0000,wsj,Democratic Sen. Amy Klobuchar of Minnesota ann...,https://www.wsj.com/politics/elections/sen-amy...
307,73,2026-01-29-12-00-01 +0000,startribune,AI image of Alex Pretti’s killing is the lates...,https://www.startribune.com/ai-image-of-alex-p...
193,59,2026-01-29-18-45-03 +0000,nypost,"Starbucks CEO lays out long-term growth plan, ...",https://nypost.com/2026/01/29/business/starbuc...
282,50,2026-01-29-14-00-00 +0000,latimes,"San José Mayor Matt Mahan, a Newsom critic and...",https://www.latimes.com/california/story/2026-...
372,49,2026-01-29-05-47-03 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...
304,44,2026-01-29-12-20-47 +0000,bbc,Nicki Minaj calls herself Trump's 'number one ...,https://www.bbc.com/news/articles/cp87l604nx8o...
81,43,2026-01-29-23-45-33 +0000,cbc,"Prime minister meets Heated Rivalry stars, pay...",https://www.cbc.ca/news/canada/livestory/heate...
301,38,2026-01-29-12-49-27 +0000,nypost,Sidelined Border Patrol chief Greg Bovino prai...,https://nypost.com/2026/01/29/us-news/sideline...
58,36,2026-01-30-00-17-00 +0000,wsj,Tulsi Gabbard has been investigating results o...,https://www.wsj.com/politics/elections/spy-chi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
